In [3]:
!pip install transformers sentence-transformers PyPDF2 faiss-cpu

In [4]:
import os
import PyPDF2
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [5]:
# Step 1: Install libraries (run once)
!pip install transformers sentence-transformers PyPDF2 faiss-cpu

# Step 2: Imports
import PyPDF2
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Step 3: Load your PDF
pdf_path = r"C:\Users\Suraj Garole\OneDrive\Documents\Python Notes.pdf"  # your file path
all_text = ""

with open(pdf_path, "rb") as f:
    pdf_reader = PyPDF2.PdfReader(f)
    for page in pdf_reader.pages:
        all_text += page.extract_text() + " "
 
print("PDF loaded successfully. Total characters:", len(all_text))

# Step 4: Split text into chunks
def split_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = split_text(all_text)
print(f"Total chunks created: {len(chunks)}")

# Step 5: Generate embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
print("Embeddings generated for all chunks.")

# Step 6: Build FAISS index
embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(chunk_embeddings)
print(f"FAISS index built with {index.ntotal} vectors.")

# Step 7: Load GPT model (GPT-Neo 1.3B)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=-1)  # CPU: -1, GPU: 0

# Step 8: Define Q&A function
def answer_question(query, k=3):
    # Embed the query
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    
    # Search in FAISS
    D, I = index.search(query_embedding, k)
    
    # Retrieve top-k chunks
    context = " ".join([chunks[i] for i in I[0]])
    
    # Generate answer using GPT-Neo
    prompt = f"Answer the question based on the context below:\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    
    result = generator(prompt, max_length=200, do_sample=True, top_p=0.95, top_k=50)
    return result[0]['generated_text'].split("Answer:")[-1].strip()

# Step 9: Test the Q&A system
query = "What is the purpose of the ESP system?"
answer = answer_question(query)
print("Question:", query)
print("Answer:", answer)


PDF loaded successfully. Total characters: 114
Total chunks created: 0


Batches: 0it [00:00, ?it/s]

Embeddings generated for all chunks.


IndexError: tuple index out of range

In [7]:
# ===============================
# Step 0: Install required libraries
# ===============================
!pip install transformers sentence-transformers PyPDF2 faiss-cpu ipywidgets

# ===============================
# Step 1: Imports
# ===============================
import PyPDF2
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output

# ===============================
# Step 2: Load your PDF
# ===============================
pdf_path = r"C:\Users\Suraj Garole\OneDrive\Documents\Python Notes.pdf"  
all_text = ""

with open(pdf_path, "rb") as f:
    pdf_reader = PyPDF2.PdfReader(f)
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:
            all_text += page_text + " "

print("PDF loaded successfully. Total characters:", len(all_text))

# ===============================
# Step 3: Split text into chunks
# ===============================
def split_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = split_text(all_text)
print(f"Total chunks created: {len(chunks)}")

# ===============================
# Step 4: Generate embeddings
# ===============================
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

# Ensure embeddings are 2D (FAISS requirement)
if len(chunk_embeddings.shape) == 1:
    chunk_embeddings = np.expand_dims(chunk_embeddings, axis=0)

print("Embeddings generated for all chunks.")

# ===============================
# Step 5: Build FAISS index
# ===============================
embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(chunk_embeddings)
print(f"FAISS index built with {index.ntotal} vectors.")

# ===============================
# Step 6: Load GPT-Neo for answer generation
# ===============================
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=-1)  # CPU=-1, GPU=0

# ===============================
# Step 7: Define Q&A function
# ===============================
def answer_question(query, k=3):
    # Embed the query
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    
    # FAISS search
    D, I = index.search(query_embedding, k)
    
    # Retrieve top-k chunks
    context = " ".join([chunks[i] for i in I[0]])
    
    # GPT-Neo prompt
    prompt = f"Answer the question based on the context below:\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    
    result = generator(prompt, max_length=200, do_sample=True, top_p=0.95, top_k=50)
    return result[0]['generated_text'].split("Answer:")[-1].strip()

# ===============================
# Step 8: Interactive Q&A in Jupyter
# ===============================
# Textbox for question
question_box = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Question:',
    layout=widgets.Layout(width='80%')
)

# Button to submit question
submit_button = widgets.Button(description="Ask AI")

# Output area
output_area = widgets.Output(layout={'border': '1px solid black'})

# Function to handle button click
def on_submit_button_clicked(b):
    with output_area:
        clear_output()
        query = question_box.value
        if query.strip() == "":
            print("Please type a question!")
            return
        print("Question:", query)
        answer = answer_question(query)
        print("Answer:", answer)

submit_button.on_click(on_submit_button_clicked)

# Display interface
display(question_box, submit_button, output_area)


PDF loaded successfully. Total characters: 0
Total chunks created: 0


Batches: 0it [00:00, ?it/s]

Embeddings generated for all chunks.
FAISS index built with 1 vectors.


Device set to use cpu


Text(value='', description='Question:', layout=Layout(width='80%'), placeholder='Type your question here...')

Button(description='Ask AI', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [8]:
import pickle

# Save chunks
with open("chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

# Save embeddings
np.save("chunk_embeddings.npy", chunk_embeddings)

print("Knowledge base saved successfully!")

# --- To load later ---
# with open("chunks.pkl", "rb") as f:
#     chunks = pickle.load(f)
# chunk_embeddings = np.load("chunk_embeddings.npy")


Knowledge base saved successfully!


In [9]:
chat_history = []

def answer_question_with_history(query, k=3):
    answer = answer_question(query, k)
    chat_history.append({"Question": query, "Answer": answer})
    return answer

# Optional: display session history
def show_chat_history():
    for entry in chat_history:
        print("Q:", entry["Question"])
        print("A:", entry["Answer"])
        print("-"*50)


In [10]:
def answer_question(query, k=3):
    # Embed the query
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    
    # Ensure query_embedding is 2D
    if len(query_embedding.shape) == 1:
        query_embedding = np.expand_dims(query_embedding, axis=0)
    
    # FAISS search
    D, I = index.search(query_embedding, k)
    
    # Retrieve top-k chunks
    context = " ".join([chunks[i] for i in I[0]])
    
    # GPT-Neo prompt
    prompt = f"Answer the question based on the context below:\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    
    result = generator(prompt, max_length=200, do_sample=True, top_p=0.95, top_k=50)
    return result[0]['generated_text'].split("Answer:")[-1].strip()



In [12]:
!pip install SpeechRecognition pyttsx3 pyaudio


   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.9 MB ? eta -:--:--
    --------------------------------------- 0.5/32.9 MB 458.6 kB/s eta 0:01:11
    --------------------------------------- 0.5/32.9 MB 458.6 kB/s eta 0:01:11
    --------------------------------------- 0.8/32.9 MB 516.3 kB/s eta 0:01:03
    --------------------------------------- 0.8/32.9 MB 516.3 kB/s eta 0:01:03
   - -------------------------------------- 1.0/32.9 MB 557.8 kB/s eta 0:00:58
   - -------------------------------------- 1.3/32.9 MB 599.1 kB/s eta 0:00:53
   - -------------------------------------- 1.3/32.9 MB 599.1 kB/s eta 0:00:53
   -- ------------------------------------- 1.8/32.9 MB 585.2 kB/s eta 0:00:54
   -- ------------------------------------- 2.1/32.9 MB 579.2 kB/s eta 0:00:54
   -- ---

In [13]:
pip install PyAudio-0.2.11-cp310-cp310-win_amd64.whl

Note: you may need to restart the kernel to use updated packages.


ERROR: PyAudio-0.2.11-cp310-cp310-win_amd64.whl is not a supported wheel on this platform.


In [14]:
import speech_recognition as sr
import pyttsx3

# Initialize
recognizer = sr.Recognizer()
engine = pyttsx3.init()

try:
    with sr.Microphone() as source:
        print("🎤 Ask your question:")
        recognizer.adjust_for_ambient_noise(source, duration=1)  # reduces noise
        audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)
        
        try:
            query = recognizer.recognize_google(audio, language="en-IN")  # Indian English
            print("✅ You asked:", query)
        except sr.UnknownValueError:
            print("❌ Sorry, could not understand speech. Try again.")
            query = None
        except sr.RequestError as e:
            print(f"❌ Could not request results; {e}")
            query = None

    if query:
        # Use your PDF Q&A function here
        answer = answer_question(query)
        print("💡 Answer:", answer)
        engine.say(answer)
        engine.runAndWait()

except sr.WaitTimeoutError:
    print("⌛ Listening timed out. Please speak clearly.")


🎤 Ask your question:
❌ Sorry, could not understand speech. Try again.


In [15]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Radio button to choose input type
input_type = widgets.RadioButtons(
    options=['Text', 'Voice'],
    description='Input Type:',
    layout=widgets.Layout(width='50%')
)

# Textbox for questions
question_box = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Question:',
    layout=widgets.Layout(width='80%')
)

# Submit button
submit_button = widgets.Button(description="Ask AI")

# Output area
output_area = widgets.Output(layout={'border': '1px solid black'})

# Handle button click
def on_submit(b):
    with output_area:
        clear_output()
        if input_type.value == 'Text':
            query = question_box.value
        else:
            # Voice input
            import speech_recognition as sr
            recognizer = sr.Recognizer()
            engine = pyttsx3.init()
            try:
                with sr.Microphone() as source:
                    print("Listening...")
                    recognizer.adjust_for_ambient_noise(source, duration=1)
                    audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)
                    try:
                        query = recognizer.recognize_google(audio)
                        print("You asked:", query)
                    except sr.UnknownValueError:
                        print("Sorry, I could not understand your speech. Please try again.")
                        return
            except sr.WaitTimeoutError:
                print("Listening timed out. Please speak clearly.")
                return
        
        if query.strip() == "":
            print("Please enter a question!")
            return
        
        answer = answer_question(query)
        print("Answer:", answer)
        if input_type.value == 'Voice':
            engine.say(answer)
            engine.runAndWait()

submit_button.on_click(on_submit)

display(input_type, question_box, submit_button, output_area)


RadioButtons(description='Input Type:', layout=Layout(width='50%'), options=('Text', 'Voice'), value='Text')

Text(value='', description='Question:', layout=Layout(width='80%'), placeholder='Type your question here...')

Button(description='Ask AI', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [16]:
import gradio as gr
iface = gr.Interface(fn=answer_question,
                     inputs="text",
                     outputs="text",
                     title="PDF Q&A Chatbot")
iface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [17]:
pip install gradio


Note: you may need to restart the kernel to use updated packages.


In [20]:
# gradio_pdf_qa_fixed.py
# Single-file, robust PDF → FAISS → Gradio Q&A app.
# Usage: python gradio_pdf_qa_fixed.py
# (or run cells in Jupyter after removing the __main__ guard)

import os
import sys
import traceback
import numpy as np
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import gradio as gr

# ---------- CONFIG ----------
pdf_path = r"C:\Users\Suraj Garole\OneDrive\Documents\Python Notes.pdf"   # <-- fixed path
chunk_size = 500
overlap = 50
embedder_name = "all-MiniLM-L6-v2"
# ----------------------------

def extract_text_pypdf(path):
    """Try extracting text using PyPDF2."""
    text = ""
    try:
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for i, page in enumerate(reader.pages):
                try:
                    ptext = page.extract_text()
                except Exception:
                    ptext = None
                if ptext:
                    text += ptext + " "
    except Exception as e:
        print("PyPDF2 error:", e)
    return text.strip()

def extract_text_pdfplumber(path):
    """Fallback using pdfplumber if available."""
    try:
        import pdfplumber
    except Exception:
        return ""
    text = ""
    try:
        with pdfplumber.open(path) as pdf:
            for p in pdf.pages:
                ptext = p.extract_text()
                if ptext:
                    text += ptext + " "
    except Exception as e:
        print("pdfplumber error:", e)
    return text.strip()

def extract_text_or_warn(path):
    """Try multiple extractors and return text (or empty)."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"PDF not found: {path}")
    print("Extracting text with PyPDF2...")
    text = extract_text_pypdf(path)
    if text:
        print("Text extracted with PyPDF2 (length:", len(text), ")")
        return text
    print("PyPDF2 found no extractable text. Trying pdfplumber (if installed)...")
    text = extract_text_pdfplumber(path)
    if text:
        print("Text extracted with pdfplumber (length:", len(text), ")")
        return text
    # No text found
    print("\nWarning: No selectable text was extracted from the PDF.")
    print("This may be a scanned PDF (images) and requires OCR (pytesseract + pdf2image) or manual text.")
    print("You can install pdfplumber (pip install pdfplumber) or use OCR. Exiting build step.\n")
    return ""

def split_text_to_chunks(text, chunk_size=500, overlap=50):
    if not text:
        return []
    words = text.split()
    if len(words) == 0:
        return []
    step = max(1, chunk_size - overlap)
    chunks = []
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i+chunk_size]).strip()
        if chunk:
            chunks.append(chunk)
    return chunks

# ---------- Build KB ----------
print("Loading PDF and building knowledge base...")
try:
    all_text = extract_text_or_warn(pdf_path)
except FileNotFoundError as e:
    print("ERROR:", e)
    sys.exit(1)
except Exception:
    print("Unexpected error while loading PDF:")
    traceback.print_exc()
    sys.exit(1)

chunks = split_text_to_chunks(all_text, chunk_size=chunk_size, overlap=overlap)
print("Total chunks created:", len(chunks))

# If no chunks, we will still launch Gradio but return helpful message in responses.
index = None
embedder = None
chunk_embeddings = None
embedding_dim = 0

if len(chunks) == 0:
    print("No chunks available. The app will run but cannot answer from the PDF until text is available.")
else:
    # Create embedder and embeddings
    try:
        print("Loading embedder:", embedder_name)
        embedder = SentenceTransformer(embedder_name)
        chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
        # If a single chunk, ensure embeddings are 2D
        if chunk_embeddings.ndim == 1:
            chunk_embeddings = np.expand_dims(chunk_embeddings, 0)
        # Safeguard: no-zero dim
        if chunk_embeddings.size == 0 or chunk_embeddings.shape[1] == 0:
            print("Error: embeddings seem empty or zero-dimension. Aborting index build.")
            chunk_embeddings = None
        else:
            # Ensure float32 contiguous array for FAISS
            chunk_embeddings = np.ascontiguousarray(chunk_embeddings.astype('float32'))
            embedding_dim = chunk_embeddings.shape[1]
            print("chunk_embeddings.shape =", chunk_embeddings.shape)
            # Build FAISS index
            index = faiss.IndexFlatL2(embedding_dim)
            index.add(chunk_embeddings)
            print(f"Built FAISS index. index.d = {index.d}, index.ntotal = {index.ntotal}")
    except Exception as e:
        print("Error while creating embeddings or building FAISS index:")
        traceback.print_exc()
        index = None

# ---------- Answer function ----------
def answer_question(query, k=3):
    """Return top-k context chunks or an instructional error message."""
    if index is None or embedder is None or len(chunks) == 0:
        return (
            "Knowledge base is not available. Possible reasons:\n"
            "- PDF had no selectable text (scanned images). Try installing pdfplumber or OCR.\n"
            "- Embeddings/index failed to build.\n\n"
            "Fix: ensure the PDF contains selectable text, install pdfplumber (pip install pdfplumber) "
            "or provide a TXT file. Then re-run the KB build."
        )
    try:
        q_emb = embedder.encode([query], convert_to_numpy=True)
        if q_emb.ndim == 1:
            q_emb = q_emb.reshape(1, -1)
        q_emb = np.ascontiguousarray(q_emb.astype('float32'))
        if q_emb.shape[1] != index.d:
            return f"Embedding dimension mismatch: query vector dim {q_emb.shape[1]} != index dim {index.d}. Rebuild index with the same embedder."
        # search
        D, I = index.search(q_emb, k)
        top_chunks = []
        for idx in I[0]:
            if 0 <= idx < len(chunks):
                top_chunks.append(chunks[idx])
        if not top_chunks:
            return "No relevant context found."
        # join with separators to show context
        context = "\n\n---\n\n".join(top_chunks)
        # For speed/safety we return context. Optionally replace this with an LLM call that uses `context`.
        return f"Top {len(top_chunks)} context chunks (used for answering):\n\n{context}"
    except Exception as e:
        traceback.print_exc()
        return f"Error during search/generation: {e}"

# ---------- Launch Gradio with auto-port fallback ----------
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about the PDF"),
    outputs="text",
    title="PDF Q&A (FAISS + SentenceTransformers)",
    allow_flagging="never"
)

def launch_gradio_with_port_fallback(interface, start_port=7860, max_tries=16, share=False):
    port = start_port
    for i in range(max_tries):
        try:
            print(f"Attempting to launch Gradio on 127.0.0.1:{port} (try {i+1}/{max_tries}) ...")
            interface.launch(server_name="127.0.0.1", server_port=port, share=share, prevent_thread_lock=True)
            return
        except OSError as e:
            print(f"Port {port} not available: {e}")
            port += 1
    raise OSError(f"Could not find an open port between {start_port} and {port-1}.")

if __name__ == "__main__":
    try:
        launch_gradio_with_port_fallback(iface, start_port=7860, max_tries=16, share=False)
    except Exception as e:
        print("Failed to launch Gradio:", e)
        traceback.print_exc()


Loading PDF and building knowledge base...
Extracting text with PyPDF2...
PyPDF2 found no extractable text. Trying pdfplumber (if installed)...

This may be a scanned PDF (images) and requires OCR (pytesseract + pdf2image) or manual text.
You can install pdfplumber (pip install pdfplumber) or use OCR. Exiting build step.

Total chunks created: 0
No chunks available. The app will run but cannot answer from the PDF until text is available.


C:\Users\Suraj Garole\Conda\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Attempting to launch Gradio on 127.0.0.1:7860 (try 1/16) ...
Port 7860 not available: Cannot find empty port in range: 7860-7860. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` parameter to `launch()`.
Attempting to launch Gradio on 127.0.0.1:7861 (try 2/16) ...
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [21]:
pip install pdfplumber pytesseract pillow


Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install python-docx sentence-transformers faiss-cpu gradio


Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install pdfplumber


Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np
import faiss
import gradio as gr
import pdfplumber
from sentence_transformers import SentenceTransformer

# 1️⃣ Load the PDF text using pdfplumber
pdf_path = r"C:\Users\Suraj Garole\Downloads\CPO 2022 All Vocab with Hindi meaning by Prashant Sir (1).pdf"

full_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            full_text += text + "\n"

if not full_text.strip():
    raise ValueError("❌ No text extracted from the PDF. It might be scanned images.")

print("✅ PDF loaded successfully. Total characters:", len(full_text))

# 2️⃣ Split into ~500-character chunks
chunk_size = 500
chunks = [full_text[i:i + chunk_size] for i in range(0, len(full_text), chunk_size)]
print(f"✅ Total chunks created: {len(chunks)}")

# 3️⃣ Encode with a sentence transformer
print("🔄 Generating embeddings...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
print("✅ Embeddings generated. Shape:", embeddings.shape)

# 4️⃣ Build a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
print(f"✅ FAISS index built with {index.ntotal} vectors.")

# 5️⃣ Define Q&A function
def answer_question(query, k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(np.array(q_emb, dtype="float32"), k)
    context = " ".join([chunks[i] for i in I[0]])
    return f"💡 Answer (context-based):\n\n{context}"

# 6️⃣ Launch Gradio
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about the PDF content"),
    outputs="text",
    title="📘 PDF Q&A Chatbot"
)

iface.launch(server_name="127.0.0.1")


✅ PDF loaded successfully. Total characters: 21144
✅ Total chunks created: 43
🔄 Generating embeddings...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Embeddings generated. Shape: (43, 384)
✅ FAISS index built with 43 vectors.
* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [28]:
# gradio_multi_doc_qa.py
import os
import pickle
import traceback
import numpy as np
import gradio as gr
import faiss
from sentence_transformers import SentenceTransformer
from docx import Document
import PyPDF2

# Optional pdfplumber for better PDF extraction
try:
    import pdfplumber
    HAS_PDFPLUMBER = True
except Exception:
    HAS_PDFPLUMBER = False

# Optional OpenAI use
try:
    import openai
    HAS_OPENAI = True
except Exception:
    HAS_OPENAI = False

# ---------- Config ----------
EMBEDDER_NAME = "all-MiniLM-L6-v2"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
PERSIST_INDEX_FILE = "faiss_index.index"
PERSIST_CHUNKS_FILE = "chunks.pkl"
PERSIST_EMB_FILE = "chunk_embeddings.npy"

# ---------- Globals ----------
embedder = None
index = None
chunks = []

# ---------- Helpers: extract text ----------
def extract_text_from_docx(path):
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs if p.text and p.text.strip()])

def extract_text_from_pdf(path):
    text = ""
    try:
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for p in reader.pages:
                try:
                    page_text = p.extract_text()
                except Exception:
                    page_text = None
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print("PyPDF2 read error:", e)

    # fallback to pdfplumber if PyPDF2 gave nothing and pdfplumber is available
    if (not text) and HAS_PDFPLUMBER:
        try:
            with pdfplumber.open(path) as pdf:
                for p in pdf.pages:
                    ptext = p.extract_text()
                    if ptext:
                        text += ptext + "\n"
        except Exception as e:
            print("pdfplumber error:", e)

    return text

def extract_text_from_txt(path):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    except Exception:
        return ""

def extract_text(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".docx":
        return extract_text_from_docx(path)
    if ext == ".pdf":
        return extract_text_from_pdf(path)
    if ext in (".txt", ".md"):
        return extract_text_from_txt(path)
    return ""  # unsupported file type

# ---------- Helpers: split & build ----------
def split_text_to_chunks(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    if not text:
        return []
    words = text.split()
    step = max(1, chunk_size - overlap)
    chunks_out = []
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i+chunk_size]).strip()
        if chunk:
            chunks_out.append(chunk)
    return chunks_out

def build_kb_from_file_paths(file_paths):
    """
    file_paths: list of filesystem paths
    returns: status string
    """
    global embedder, index, chunks

    # ensure embedder loaded
    if embedder is None:
        embedder = SentenceTransformer(EMBEDDER_NAME)

    texts = []
    for p in file_paths:
        try:
            t = extract_text(p)
            if t and t.strip():
                texts.append(t)
        except Exception as e:
            print("Error extracting", p, e)

    if not texts:
        return "No selectable text extracted from uploaded files. Try text-based DOCX/TXT or searchable PDFs."

    # combine and chunk
    full_text = "\n\n".join(texts)
    chunks = split_text_to_chunks(full_text)
    if not chunks:
        return "Failed to create text chunks from documents."

    # embeddings
    try:
        embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
    except Exception as e:
        traceback.print_exc()
        return f"Error while encoding embeddings: {e}"

    # shape fixes & dtype
    if embeddings.ndim == 1:
        embeddings = np.expand_dims(embeddings, 0)
    embeddings = np.ascontiguousarray(embeddings.astype("float32"))

    if embeddings.size == 0 or embeddings.shape[1] == 0:
        return "Embeddings empty or invalid dimension."

    # build faiss index
    embedding_dim = embeddings.shape[1]
    idx = faiss.IndexFlatL2(embedding_dim)
    idx.add(embeddings)

    # persist
    try:
        faiss.write_index(idx, PERSIST_INDEX_FILE)
        with open(PERSIST_CHUNKS_FILE, "wb") as f:
            pickle.dump(chunks, f)
        np.save(PERSIST_EMB_FILE, embeddings)
    except Exception as e:
        print("Warning: could not persist index to disk:", e)

    # set globals
    index = idx

    return f"KB built: {len(chunks)} chunks, embedding_dim={embedding_dim}"

# ---------- Load persisted KB if present ----------
def try_load_persisted_kb():
    global embedder, index, chunks
    if embedder is None:
        embedder = SentenceTransformer(EMBEDDER_NAME)
    if os.path.exists(PERSIST_INDEX_FILE) and os.path.exists(PERSIST_CHUNKS_FILE):
        try:
            index = faiss.read_index(PERSIST_INDEX_FILE)
            with open(PERSIST_CHUNKS_FILE, "rb") as f:
                chunks = pickle.load(f)
            return True
        except Exception as e:
            print("Failed to load persisted KB:", e)
    return False

# ---------- Answering ----------
def answer_question(query, top_k=3, use_openai=False):
    """
    Returns either the top-k context chunks OR (optionally) uses OpenAI to generate a final answer.
    """
    global index, chunks, embedder

    # ensure embedder
    if embedder is None:
        embedder = SentenceTransformer(EMBEDDER_NAME)

    # attempt to load persisted KB if not in memory
    if (index is None or not chunks) and not try_load_persisted_kb():
        return "Knowledge base not ready. Upload files and click 'Build KB' first."

    # embed query
    q_emb = embedder.encode([query], convert_to_numpy=True)
    if q_emb.ndim == 1:
        q_emb = q_emb.reshape(1, -1)
    q_emb = np.ascontiguousarray(q_emb.astype("float32"))

    # sanity checks
    if q_emb.shape[1] != index.d:
        return f"Embedding dimension mismatch: query dim {q_emb.shape[1]} vs index dim {index.d}. Rebuild KB."

    # search
    D, I = index.search(q_emb, min(top_k, index.ntotal))
    top_chunks = []
    for idx in I[0]:
        if 0 <= idx < len(chunks):
            top_chunks.append(chunks[idx])
    if not top_chunks:
        return "No relevant context found."

    context = "\n\n---\n\n".join(top_chunks)

    # Optionally use OpenAI to produce a final answer (requires OPENAI_API_KEY and openai installed)
    if use_openai:
        if not HAS_OPENAI:
            return "OpenAI package not installed. Install with `pip install openai` to use generation."
        key = os.getenv("OPENAI_API_KEY") or ""
        if not key:
            return "OPENAI_API_KEY env var not set. Set it to use OpenAI generation."
        try:
            openai.api_key = key
            prompt = f"Use the provided context to answer the question.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
            resp = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role":"user","content":prompt}],
                max_tokens=300,
                temperature=0.2
            )
            return resp.choices[0].message.content.strip()
        except Exception as e:
            traceback.print_exc()
            return f"OpenAI error: {e}"

    # default: return the retrieved context so user can read the evidence
    return f"Top {len(top_chunks)} context chunks (evidence):\n\n{context}"

# ---------- Gradio callbacks ----------
def on_build_kb(gr_files):
    """
    gr_files is a list of uploaded file objects (each has .name attribute pointing to temp file path).
    """
    if not gr_files:
        return "No files provided. Upload DOCX, PDF, or TXT files."
    file_paths = []
    for f in gr_files:
        # f is a tempfile object; get its path
        path = getattr(f, "name", None) or f
        file_paths.append(path)
    return build_kb_from_file_paths(file_paths)

def on_ask(query, top_k, use_openai):
    if not query or not query.strip():
        return "Please enter a question."
    return answer_question(query.strip(), top_k=top_k, use_openai=use_openai)

# ---------- Gradio UI ----------
with gr.Blocks(title="Multi-file Q&A (DOCX/PDF/TXT)") as demo:
    gr.Markdown("## Upload documents (DOCX / PDF / TXT) and click **Build KB**")
    file_input = gr.File(file_count="multiple", label="Upload files (DOCX / PDF / TXT)")
    build_btn = gr.Button("Build KB")
    build_out = gr.Textbox(label="Build status", interactive=False)

    gr.Markdown("## Ask questions (uses the knowledge base)")
    query = gr.Textbox(label="Question", placeholder="Type your question here...")
    top_k = gr.Slider(minimum=1, maximum=5, step=1, value=3, label="Top-k context chunks")
    use_openai_chk = gr.Checkbox(value=False, label="Use OpenAI to generate final answer (optional)")

    ask_btn = gr.Button("Ask")
    answer_out = gr.Textbox(label="Answer / Retrieved context", interactive=False)

    build_btn.click(fn=on_build_kb, inputs=file_input, outputs=build_out)
    ask_btn.click(fn=on_ask, inputs=[query, top_k, use_openai_chk], outputs=answer_out)

# If run as script, try to auto-load persisted KB
try_load_persisted_kb()

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [29]:
"""
qa_with_ocr_and_local_llm.py

Features:
- Upload DOCX/PDF/TXT files (Gradio)
- Extract text (PyPDF2/pdfplumber) + OCR fallback (pytesseract/pdf2image or ocr.space)
- Build FAISS index (SentenceTransformer embeddings)
- Answer queries; options: 'none' (show contexts), 'openai' (requires OPENAI_API_KEY), 'local' (offline Flan-T5)
"""

import os
import pickle
import traceback
import numpy as np
import gradio as gr
import faiss
from sentence_transformers import SentenceTransformer
from docx import Document
import PyPDF2
import time

# Optional modules (import inside try blocks)
try:
    import pdfplumber
    HAS_PDFPLUMBER = True
except Exception:
    HAS_PDFPLUMBER = False

try:
    import pdf2image
    HAS_PDF2IMAGE = True
except Exception:
    HAS_PDF2IMAGE = False

try:
    import pytesseract
    HAS_PYTESSERACT = True
except Exception:
    HAS_PYTESSERACT = False

try:
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
    HAS_TRANSFORMERS = True
except Exception:
    HAS_TRANSFORMERS = False

try:
    import openai
    HAS_OPENAI = True
except Exception:
    HAS_OPENAI = False

# ----------------- CONFIG (edit these) -----------------
EMBEDDER_NAME = "all-MiniLM-L6-v2"
CHUNK_SIZE = 400          # smaller chunk size often works better for LLM context
CHUNK_OVERLAP = 50
PERSIST_INDEX_FILE = "faiss_index.index"
PERSIST_CHUNKS_FILE = "chunks.pkl"
PERSIST_EMB_FILE = "chunk_embeddings.npy"

# OCR options: "tesseract", "ocrspace", or "none"
OCR_MODE = "tesseract"   # set "ocrspace" to use ocr.space API or "none" to disable OCR fallback
# If OCR_MODE == "ocrspace" set OCRSPACE_API_KEY to your key (or leave empty to use free limited usage)
OCRSPACE_API_KEY = ""     # e.g. "helloworld" or your key

# If using local Tesseract or Poppler, set these if not on PATH
TESSERACT_CMD = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # set if needed, else leave blank
POPPLER_PATH = r"C:\Program Files\poppler-22.04.0\Library\bin"    # set path to poppler bin on Windows (if needed)

# Local LLM options (offline generator)
LOCAL_MODEL_NAME = "google/flan-t5-small"  # "flan-t5-small" or "flan-t5-base"
USE_CUDA_IF_AVAILABLE = False              # set True if you have GPU and torch+cuda installed
# -------------------------------------------------------

# Globals
embedder = None
index = None
chunks = []
local_tokenizer = None
local_model = None

# ---------- text extraction ----------
def extract_text_from_docx(path):
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs if p.text and p.text.strip()])

def extract_text_from_pdf(path):
    text = ""
    # try PyPDF2
    try:
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for p in reader.pages:
                try:
                    page_text = p.extract_text()
                except Exception:
                    page_text = None
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print("PyPDF2 read error:", e)

    # fallback to pdfplumber if available and no text
    if (not text) and HAS_PDFPLUMBER:
        try:
            with pdfplumber.open(path) as pdf:
                for p in pdf.pages:
                    ptext = p.extract_text()
                    if ptext:
                        text += ptext + "\n"
        except Exception as e:
            print("pdfplumber error:", e)

    # OCR fallback if still empty and OCR_MODE configured
    if (not text) and OCR_MODE != "none":
        print("No selectable text; attempting OCR fallback (OCR_MODE=%s)..." % OCR_MODE)
        if OCR_MODE == "tesseract":
            if not HAS_PDF2IMAGE or not HAS_PYTESSERACT:
                print("pdf2image or pytesseract not installed. Install pdf2image & pytesseract to use local OCR.")
            else:
                # optionally set tesseract cmd
                if TESSERACT_CMD:
                    pytesseract.pytesseract.tesseract_cmd = TESSERACT_CMD
                # convert pages to images and OCR
                try:
                    images = pdf2image.convert_from_path(path, dpi=300, poppler_path=POPPLER_PATH if POPPLER_PATH else None)
                    for img in images:
                        ocr_text = pytesseract.image_to_string(img)
                        if ocr_text:
                            text += ocr_text + "\n"
                except Exception as e:
                    print("Local OCR error:", e, "Make sure poppler and tesseract are installed if using local OCR.")
        elif OCR_MODE == "ocrspace":
            # Use ocr.space API
            try:
                import requests, base64
                with open(path, "rb") as f:
                    files = {"file": f}
                    data = {"apikey": OCRSPACE_API_KEY, "language": "eng"}
                    resp = requests.post("https://api.ocr.space/parse/image", files=files, data=data)
                    result = resp.json()
                    if result.get("ParsedResults"):
                        text = "\n".join([pr.get("ParsedText","") for pr in result["ParsedResults"]])
            except Exception as e:
                print("ocr.space API error:", e)
    return text

def extract_text_from_txt(path):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    except Exception:
        return ""

def extract_text(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".docx":
        return extract_text_from_docx(path)
    if ext == ".pdf":
        return extract_text_from_pdf(path)
    if ext in (".txt", ".md"):
        return extract_text_from_txt(path)
    return ""

# ---------- chunking & KB build ----------
def split_text_to_chunks(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    if not text:
        return []
    words = text.split()
    step = max(1, chunk_size - overlap)
    chunks_out = []
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i+chunk_size]).strip()
        if chunk:
            chunks_out.append(chunk)
    return chunks_out

def build_kb_from_paths(paths):
    global embedder, index, chunks
    if embedder is None:
        embedder = SentenceTransformer(EMBEDDER_NAME)

    texts = []
    for p in paths:
        try:
            t = extract_text(p)
            if t and t.strip():
                texts.append(t)
        except Exception as e:
            print("Error extracting", p, e)

    if not texts:
        return "No selectable text extracted. Provide DOCX/TXT or searchable PDF, or enable OCR."

    full_text = "\n\n".join(texts)
    chunks = split_text_to_chunks(full_text)
    if not chunks:
        return "Failed to create chunks from text."

    embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
    if embeddings.ndim == 1:
        embeddings = np.expand_dims(embeddings, 0)
    embeddings = np.ascontiguousarray(embeddings.astype("float32"))

    if embeddings.size == 0 or embeddings.shape[1] == 0:
        return "Embeddings empty or invalid."

    idx = faiss.IndexFlatL2(embeddings.shape[1])
    idx.add(embeddings)

    # persist
    try:
        faiss.write_index(idx, PERSIST_INDEX_FILE)
        with open(PERSIST_CHUNKS_FILE, "wb") as f:
            pickle.dump(chunks, f)
        np.save(PERSIST_EMB_FILE, embeddings)
    except Exception as e:
        print("Warning: could not persist index:", e)

    index = idx
    return f"KB built: {len(chunks)} chunks (embedding_dim={embeddings.shape[1]})."

# ---------- try to load persisted KB ----------
def try_load_kb():
    global embedder, index, chunks
    if embedder is None:
        embedder = SentenceTransformer(EMBEDDER_NAME)
    if os.path.exists(PERSIST_INDEX_FILE) and os.path.exists(PERSIST_CHUNKS_FILE):
        try:
            index = faiss.read_index(PERSIST_INDEX_FILE)
            with open(PERSIST_CHUNKS_FILE, "rb") as f:
                chunks = pickle.load(f)
            print("Loaded persisted KB:", len(chunks), "chunks.")
            return True
        except Exception as e:
            print("Failed loading persisted KB:", e)
    return False

# ---------- local LLM (Flan-T5) lazy load ----------
def load_local_llm():
    global local_model, local_tokenizer
    if not HAS_TRANSFORMERS:
        raise RuntimeError("transformers library not installed.")
    if local_model is None or local_tokenizer is None:
        print("Loading local model:", LOCAL_MODEL_NAME)
        local_tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_NAME)
        local_model = AutoModelForSeq2SeqLM.from_pretrained(LOCAL_MODEL_NAME)
        # optionally move to cuda
        if USE_CUDA_IF_AVAILABLE:
            try:
                import torch
                if torch.cuda.is_available():
                    local_model = local_model.to('cuda')
            except Exception as e:
                print("CUDA move error:", e)

def generate_with_local_model(context, question, max_new_tokens=200):
    load_local_llm()
    prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    inputs = local_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    if USE_CUDA_IF_AVAILABLE:
        import torch
        if torch.cuda.is_available():
            inputs = {k: v.to('cuda') for k, v in inputs.items()}
            out = local_model.generate(**inputs, max_new_tokens=max_new_tokens)
            out_text = local_tokenizer.decode(out[0], skip_special_tokens=True)
            return out_text
    out = local_model.generate(**inputs, max_new_tokens=max_new_tokens)
    out_text = local_tokenizer.decode(out[0], skip_special_tokens=True)
    return out_text

# ---------- answering ----------
def answer_question(query, top_k=3, generation_mode="none"):
    """
    generation_mode: "none" (just return contexts), "openai", "local"
    """
    global embedder, index, chunks
    if embedder is None:
        embedder = SentenceTransformer(EMBEDDER_NAME)

    # try loading existing KB if not in memory
    if (index is None or not chunks) and not try_load_kb():
        return "KB not ready. Upload files and click 'Build KB' first."

    # embed query
    q_emb = embedder.encode([query], convert_to_numpy=True)
    if q_emb.ndim == 1:
        q_emb = q_emb.reshape(1, -1)
    q_emb = np.ascontiguousarray(q_emb.astype("float32"))

    if q_emb.shape[1] != index.d:
        return f"Embedding dimension mismatch: query dim {q_emb.shape[1]} vs index dim {index.d}. Rebuild KB."

    D, I = index.search(q_emb, min(top_k, index.ntotal))
    top_chunks = []
    for idx in I[0]:
        if 0 <= idx < len(chunks):
            top_chunks.append(chunks[idx])

    if not top_chunks:
        return "No relevant context found."

    context = "\n\n---\n\n".join(top_chunks)

    # generation modes
    if generation_mode == "none":
        return f"Top {len(top_chunks)} context chunks:\n\n{context}"
    elif generation_mode == "openai":
        if not HAS_OPENAI:
            return "OpenAI not installed. Install openai package to use this mode."
        key = os.getenv("OPENAI_API_KEY", "")
        if not key:
            return "OPENAI_API_KEY not set (environment variable). Set it to use OpenAI."
        try:
            openai.api_key = key
            prompt = f"Answer the question using the context below.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
            resp = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300,
                temperature=0.2
            )
            return resp.choices[0].message.content.strip()
        except Exception as e:
            traceback.print_exc()
            return f"OpenAI error: {e}"
    elif generation_mode == "local":
        if not HAS_TRANSFORMERS:
            return "Transformers not installed; cannot run local generation."
        try:
            ans = generate_with_local_model(context, query, max_new_tokens=200)
            return ans
        except Exception as e:
            traceback.print_exc()
            return f"Local model error: {e}"
    else:
        return "Unknown generation_mode. Use none/openai/local."

# ---------- Gradio callbacks ----------
def on_build_kb(uploaded_files):
    if not uploaded_files:
        return "No files uploaded."
    file_paths = []
    for f in uploaded_files:
        # f is a temporary file object in Gradio, path is f.name
        p = getattr(f, "name", None) or f
        file_paths.append(p)
    return build_kb_from_paths(file_paths)

def on_ask(question, top_k, gen_mode):
    if not question or not question.strip():
        return "Please enter a question."
    return answer_question(question.strip(), top_k=top_k, generation_mode=gen_mode)

# ---------- Build Gradio UI ----------
with gr.Blocks(title="Multi-file Q&A with OCR + Local LLM") as demo:
    gr.Markdown("## Upload DOCX / PDF / TXT (PDFs may be scanned) and click **Build KB**")
    file_input = gr.File(file_count="multiple", label="Upload files")
    build_btn = gr.Button("Build KB")
    build_status = gr.Textbox(label="Build status", interactive=False)

    gr.Markdown("## Ask questions")
    question = gr.Textbox(label="Question")
    top_k = gr.Slider(minimum=1, maximum=5, step=1, value=3, label="Top-k context chunks")
    gen_mode = gr.Radio(choices=["none", "openai", "local"], value="none", label="Generation mode")
    ask_btn = gr.Button("Ask")
    answer_box = gr.Textbox(label="Answer / Context", interactive=False)

    build_btn.click(on_build_kb, inputs=file_input, outputs=build_status)
    ask_btn.click(on_ask, inputs=[question, top_k, gen_mode], outputs=answer_box)

# try to preload persisted KB if available
try_load_kb()

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [30]:
pip install pdf2image pytesseract pillow


Note: you may need to restart the kernel to use updated packages.


In [32]:
# ===============================
# Step 0: Install required libraries
# ===============================
!pip install gradio sentence-transformers faiss-cpu python-docx PyPDF2 pdfplumber

# ===============================
# Step 1: Imports
# ===============================
import pdfplumber
from docx import Document
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import gradio as gr

# ===============================
# Step 2: Load DOCX or PDF text
# ===============================
def load_file_text(file_path):
    text = ""
    if file_path.lower().endswith(".pdf"):
        # Using pdfplumber for PDFs (no OCR)
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + " "
    elif file_path.lower().endswith(".docx"):
        doc = Document(file_path)
        for para in doc.paragraphs:
            text += para.text + " "
    elif file_path.lower().endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        raise ValueError("Unsupported file type. Use PDF, DOCX, or TXT.")
    return text

# ✅ Your file path (CPO 2022 Vocabulary PDF)
file_path = r"C:\Users\Suraj Garole\Downloads\CPO 2022 All Vocab with Hindi meaning by Prashant Sir (1).pdf"

all_text = load_file_text(file_path)
print("✅ File loaded. Total characters:", len(all_text))

# ===============================
# Step 3: Split text into chunks
# ===============================
def split_text(text, chunk_size=400, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = split_text(all_text)
print(f"✅ Total chunks created: {len(chunks)}")

# ===============================
# Step 4: Generate embeddings
# ===============================
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

# Ensure 2D array
if len(chunk_embeddings.shape) == 1:
    chunk_embeddings = np.expand_dims(chunk_embeddings, axis=0)

print("✅ Embeddings generated.")

# ===============================
# Step 5: Build FAISS index
# ===============================
embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(chunk_embeddings)
print(f"✅ FAISS index built with {index.ntotal} vectors.")

# ===============================
# Step 6: Define Q&A function
# ===============================
def answer_question(query, k=3):
    # Embed query
    query_embedding = embedder.encode([query], convert_to_numpy=True).astype("float32")
    # FAISS search
    D, I = index.search(query_embedding, k)
    # Retrieve top-k chunks
    context = " ".join([chunks[i] for i in I[0]])
    return f"💡 Answer (context-based):\n\n{context}"

# ===============================
# Step 7: Gradio UI
# ===============================
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about the document"),
    outputs="text",
    title="📘 PDF Q&A Chatbot (CPO 2022 Vocabulary)"
)

iface.launch(server_name="127.0.0.1")


✅ File loaded. Total characters: 21144
✅ Total chunks created: 11


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Embeddings generated.
✅ FAISS index built with 11 vectors.
* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [34]:
# ===============================
# Step 0: Install required libraries
# ===============================
# !pip install gradio sentence-transformers faiss-cpu python-docx PyPDF2 pdfplumber transformers

# ===============================
# Step 1: Imports
# ===============================
import pdfplumber
from docx import Document
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import gradio as gr

# ===============================
# Step 2: Load DOCX or PDF text
# ===============================
def load_file_text(file_path):
    text = ""
    if file_path.lower().endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + " "
    elif file_path.lower().endswith(".docx"):
        doc = Document(file_path)
        for para in doc.paragraphs:
            text += para.text + " "
    elif file_path.lower().endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        raise ValueError("Unsupported file type. Use PDF, DOCX, or TXT.")
    return text

# ✅ Example file path (CPO 2022 Vocabulary PDF)
file_path = r"C:\Users\Suraj Garole\Downloads\CPO 2022 All Vocab with Hindi meaning by Prashant Sir (1).pdf"

all_text = load_file_text(file_path)
print("✅ File loaded. Total characters:", len(all_text))

# ===============================
# Step 3: Split text into chunks
# ===============================
def split_text(text, chunk_size=400, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = split_text(all_text)
print(f"✅ Total chunks created: {len(chunks)}")

# ===============================
# Step 4: Generate embeddings
# ===============================
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

# Ensure 2D array
if len(chunk_embeddings.shape) == 1:
    chunk_embeddings = np.expand_dims(chunk_embeddings, axis=0)

print("✅ Embeddings generated.")

# ===============================
# Step 5: Build FAISS index
# ===============================
embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(chunk_embeddings)
print(f"✅ FAISS index built with {index.ntotal} vectors.")

# ===============================
# Step 6: Load small LLM (Flan-T5) for answer generation
# ===============================
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# ===============================
# Step 7: Define Q&A function
# ===============================
def answer_question(query, k=3):
    # Embed query
    query_embedding = embedder.encode([query], convert_to_numpy=True).astype("float32")
    # FAISS search
    D, I = index.search(query_embedding, k)
    # Retrieve top-k chunks as context
    context = " ".join([chunks[i] for i in I[0]])
    
    # Prepare prompt for Flan-T5
    prompt = f"Answer the question based on the context below.\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    
    # Generate answer
    result = generator(prompt, max_length=200, do_sample=False)
    return result[0]['generated_text']

# ===============================
# Step 8: Gradio UI
# ===============================
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about the document"),
    outputs="text",
    title="📘 PDF Q&A Chatbot (Flan-T5 + FAISS)"
)

# Launch Gradio on a free port automatically
iface.launch(server_name="127.0.0.1")


✅ File loaded. Total characters: 21144
✅ Total chunks created: 11


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Embeddings generated.
✅ FAISS index built with 11 vectors.


tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\Suraj Garole\Conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Suraj Garole\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP dow

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [37]:
# ===============================
# Step 0: Install required libraries
# ===============================
!pip install gradio sentence-transformers faiss-cpu python-docx PyPDF2 pdfplumber transformers pyttsx3 SpeechRecognition pyaudio --quiet

# ===============================
# Step 1: Imports
# ===============================
import pdfplumber
from docx import Document
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import gradio as gr
import pyttsx3
import speech_recognition as sr

# ===============================
# Step 2: Load DOCX/PDF/TXT
# ===============================
def load_file_text(file_path):
    text = ""
    if file_path.lower().endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + " "
    elif file_path.lower().endswith(".docx"):
        doc = Document(file_path)
        for para in doc.paragraphs:
            text += para.text + " "
    elif file_path.lower().endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        raise ValueError("Unsupported file type. Use PDF, DOCX, or TXT.")
    return text

# ✅ Updated PDF path
file_path = r"C:\Users\Suraj Garole\Downloads\CPO 2022 All Vocab with Hindi meaning by Prashant Sir (1).pdf"

all_text = load_file_text(file_path)
print("File loaded. Total characters:", len(all_text))

# ===============================
# Step 3: Split text into chunks
# ===============================
def split_text(text, chunk_size=400, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = split_text(all_text)
print(f"Total chunks created: {len(chunks)}")

# ===============================
# Step 4: Generate embeddings
# ===============================
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

# Ensure 2D
if len(chunk_embeddings.shape) == 1:
    chunk_embeddings = np.expand_dims(chunk_embeddings, axis=0)

print("Embeddings generated.")

# ===============================
# Step 5: Build FAISS index
# ===============================
embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(chunk_embeddings)
print(f"FAISS index built with {index.ntotal} vectors.")

# ===============================
# Step 6: Load Flan-T5 LLM
# ===============================
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# ===============================
# Step 7: Setup voice engine
# ===============================
engine = pyttsx3.init()
def speak(text):
    engine.say(text)
    engine.runAndWait()

# ===============================
# Step 8: Recognize voice
# ===============================
recognizer = sr.Recognizer()
def listen_voice():
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        try:
            query = recognizer.recognize_google(audio)
            print("You said:", query)
            return query
        except sr.UnknownValueError:
            return "Sorry, I could not understand your speech."
        except sr.RequestError:
            return "Could not request results; check your internet connection."

# ===============================
# Step 9: Q&A function
# ===============================
def answer_question(query, k=3):
    query = query.strip()
    if query == "":
        return "Please ask a question."
    
    # Embed query
    query_embedding = embedder.encode([query], convert_to_numpy=True).astype("float32")
    
    # FAISS search
    D, I = index.search(query_embedding, k)
    
    # Retrieve top-k chunks
    context = " ".join([chunks[i] for i in I[0]])
    
    # Decide whether context is useful
    if len(context.strip()) < 20:
        context = ""  # No useful document info
    
    # Prompt for Flan-T5
    if context:
        prompt = f"Answer the question based on the context below.\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    else:
        prompt = f"Answer the question:\n\nQuestion: {query}\nAnswer:"
    
    # Generate answer
    result = generator(prompt, max_length=200, do_sample=False)
    answer = result[0]['generated_text']
    
    # Speak answer
    speak(answer)
    
    return answer

# ===============================
# Step 10: Gradio interface
# ===============================
def chatbot_interface(input_type, text_input):
    if input_type == "Voice":
        query = listen_voice()
    else:
        query = text_input
    answer = answer_question(query)
    return answer

iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[gr.Radio(["Text", "Voice"], label="Input Type"), gr.Textbox(lines=2, placeholder="Type your question")],
    outputs="text",
    title="Universal Q&A Chatbot (Voice + Text + PDF Knowledge)"
)

# ✅ Let Gradio select a free port automatically
iface.launch(server_name="127.0.0.1", server_port=None)


File loaded. Total characters: 21144
Total chunks created: 11


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generated.
FAISS index built with 11 vectors.


Device set to use cpu


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [39]:
import os
import threading
import pyttsx3
from sentence_transformers import SentenceTransformer
import faiss
from docx import Document
import PyPDF2
import gradio as gr

# Initialize
engine = pyttsx3.init()
model = SentenceTransformer('all-MiniLM-L6-v2')

# ----------------- TTS in background -----------------
def speak(text):
    def run():
        engine.say(text)
        engine.runAndWait()
    t = threading.Thread(target=run)
    t.start()

# ----------------- Read DOCX -----------------
def read_docx(path):
    doc = Document(path)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text

# ----------------- Read PDF (text only) -----------------
def read_pdf(path):
    text = ""
    pdf_file = open(path, 'rb')
    reader = PyPDF2.PdfReader(pdf_file)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# ----------------- Read TXT -----------------
def read_txt(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

# ----------------- Load knowledge files -----------------
knowledge_texts = []
file_paths = [
    r"C:\Users\Suraj Garole\Downloads\CPO 2022 All Vocab with Hindi meaning by Prashant Sir (1).pdf",
    # Add more files here if needed
]

for path in file_paths:
    if path.endswith('.docx'):
        knowledge_texts.append(read_docx(path))
    elif path.endswith('.pdf'):
        knowledge_texts.append(read_pdf(path))
    elif path.endswith('.txt'):
        knowledge_texts.append(read_txt(path))

# ----------------- Build FAISS index -----------------
all_text = " ".join(knowledge_texts)
chunks = [all_text[i:i+500] for i in range(0, len(all_text), 500)]
embeddings = model.encode(chunks)
dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# ----------------- Chatbot logic -----------------
def answer_question(query):
    query_vec = model.encode([query])
    D, I = index.search(query_vec, k=1)
    answer = chunks[I[0][0]]
    speak(answer)  # runs in background thread
    return answer

# ----------------- Gradio Interface -----------------
def chatbot_interface(input_type, user_input):
    if input_type == "Text":
        query = user_input
    else:
        query = user_input  # voice input placeholder
    return answer_question(query)

iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[gr.Radio(["Text", "Voice"], label="Input Type"),
            gr.Textbox(lines=2, placeholder="Type your question")],
    outputs="text",
    title="Universal Q&A Chatbot (Text + Voice + PDF Knowledge)"
)

# Launch Gradio (automatically picks free port)
iface.launch(server_name="127.0.0.1")


* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [40]:
import os
import gradio as gr
import socket

# ---------------- 1) Knowledge Folder ----------------
knowledge_folder = "knowledge"
if not os.path.exists(knowledge_folder):
    os.makedirs(knowledge_folder)
    print(f"Folder '{knowledge_folder}' created. Place your DOCX/PDF/TXT files there.")

# ---------------- 2) Dummy chatbot function ----------------
def chatbot_interface(input_type, user_input):
    return f"You said ({input_type}): {user_input}"

# ---------------- 3) Find a free port ----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))  # Bind to a free port assigned by OS
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()
print(f"Using free port: {free_port}")

# ---------------- 4) Launch Gradio ----------------
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[gr.Radio(["Text", "Voice"], label="Input Type"),
            gr.Textbox(lines=2, placeholder="Type your question here...")],
    outputs=gr.Textbox(label="Response"),
    title="Local Chatbot",
    description="Ask anything! Place DOCX/PDF/TXT files in the 'knowledge' folder to provide answers."
)

iface.launch(server_name="127.0.0.1", server_port=free_port, inbrowser=True, share=False)


Folder 'knowledge' created. Place your DOCX/PDF/TXT files there.
Using free port: 55152
* Running on local URL:  http://127.0.0.1:55152
* To create a public link, set `share=True` in `launch()`.


In [41]:
import os
import gradio as gr
import openai
import socket

# ---------------- 1) Knowledge Folder ----------------
knowledge_folder = "knowledge"
if not os.path.exists(knowledge_folder):
    os.makedirs(knowledge_folder)
    print(f"Folder '{knowledge_folder}' created. Place your DOCX/PDF/TXT files there.")

# ---------------- 2) OpenAI API Key ----------------
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your key

# ---------------- 3) GPT Chatbot Function ----------------
def chatbot_interface(input_type, user_input):
    """
    Sends user input to GPT and returns the response.
    """
    if input_type == "Voice":
        return "Voice input not implemented yet."
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # or gpt-4 if you have access
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_input}
            ],
            temperature=0.7
        )
        answer = response['choices'][0]['message']['content']
        return answer
    except Exception as e:
        return f"Error: {str(e)}"

# ---------------- 4) Find a free port ----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))  # OS assigns a free port
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()
print(f"Using free port: {free_port}")

# ---------------- 5) Launch Gradio ----------------
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[gr.Radio(["Text", "Voice"], label="Input Type"),
            gr.Textbox(lines=2, placeholder="Type your question here...")],
    outputs=gr.Textbox(label="Response"),
    title="GPT-like Chatbot",
    description="Ask anything! Place DOCX/PDF/TXT files in the 'knowledge' folder for extra info."
)

iface.launch(server_name="127.0.0.1", server_port=free_port, inbrowser=True, share=False)


Using free port: 62156
* Running on local URL:  http://127.0.0.1:62156
* To create a public link, set `share=True` in `launch()`.


In [42]:
import gradio as gr
import socket

# ----------------- 1) Chatbot function -----------------
def chatbot_interface(user_input):
    # Replace with your actual model/API
    response = f"You said: {user_input}"
    return response

# ----------------- 2) Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))  # bind to a free port assigned by the OS
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- 3) Create Gradio Interface -----------------
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(lines=3, placeholder="Ask anything..."),
    outputs="text",
    title="Auto GPT-like Chatbot",
    description="Type your question and get instant responses.",
    flagging_mode="never"
)

# ----------------- 4) Launch safely -----------------
iface.launch(
    server_name="127.0.0.1",    # localhost
    server_port=free_port,      # guaranteed free port
    inbrowser=True,             # automatically opens in browser
    prevent_thread_lock=True    # avoids WinError 10061
)


* Running on local URL:  http://127.0.0.1:55051
* To create a public link, set `share=True` in `launch()`.


In [43]:
# ----------------- 0) Imports -----------------
import gradio as gr
from transformers import pipeline
import socket

# ----------------- 1) Load GPT-style model -----------------
# We'll use a local Hugging Face model for offline capability
# Replace with your preferred model (e.g., "gpt2" or a fine-tuned GPT)
chatbot = pipeline("text-generation", model="gpt2", device=-1)  # CPU; device=0 for GPU

# ----------------- 2) Function for Q&A -----------------
def gpt_chatbot(user_input):
    try:
        output = chatbot(user_input, max_length=200, do_sample=True, temperature=0.7)
        response = output[0]["generated_text"]
        # Remove repeated user input
        response = response.replace(user_input, "").strip()
        return response
    except Exception as e:
        return f"Error: {e}"

# ----------------- 3) Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- 4) Create Gradio Interface -----------------
iface = gr.Interface(
    fn=gpt_chatbot,
    inputs=gr.Textbox(lines=3, placeholder="Ask me anything..."),
    outputs=gr.Textbox(label="GPT Response"),
    title="Local GPT Chatbot",
    description="A fully offline GPT-style Q&A chatbot.",
    flagging_mode="never"
)

# ----------------- 5) Launch -----------------
iface.launch(
    server_name="127.0.0.1",
    server_port=free_port,
    inbrowser=True,
    prevent_thread_lock=True
)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\Suraj Garole\Conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Suraj Garole\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


* Running on local URL:  http://127.0.0.1:59350
* To create a public link, set `share=True` in `launch()`.


In [44]:
# ----------------- 0) Imports -----------------
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import socket

# ----------------- 1) Load GPT model -----------------
# Using GPT-2 for demonstration; you can replace with a larger GPT model if you want
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# ----------------- 2) Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- 3) Streaming Q&A function -----------------
def gpt_chatbot_stream(user_input):
    input_ids = tokenizer.encode(user_input, return_tensors="pt")
    output_ids = model.generate(
        input_ids,
        max_length=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Remove the user input from output
    response = output_text.replace(user_input, "").strip()
    
    # Yield each word one by one for streaming effect
    for word in response.split():
        yield " ".join(response.split()[:response.split().index(word)+1])

# ----------------- 4) Create Gradio Interface -----------------
iface = gr.Interface(
    fn=gpt_chatbot_stream,
    inputs=gr.Textbox(lines=3, placeholder="Ask me anything..."),
    outputs=gr.Textbox(label="GPT Response"),
    title="Local Streaming GPT Chatbot",
    description="A fully offline GPT-style chatbot with streaming responses.",
    flagging_mode="never"
)

# ----------------- 5) Launch -----------------
iface.launch(
    server_name="127.0.0.1",
    server_port=free_port,
    inbrowser=True,
    prevent_thread_lock=True
)


* Running on local URL:  http://127.0.0.1:58502
* To create a public link, set `share=True` in `launch()`.


In [45]:
# ----------------- Imports -----------------
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import socket

# ----------------- Load Q&A GPT model -----------------
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ----------------- Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- Q&A Function -----------------
def qa_chatbot(user_input):
    # Prefix instruction for Q&A
    input_text = f"Question: {user_input} Answer:"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(
        input_ids,
        max_length=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# ----------------- Gradio Interface -----------------
iface = gr.Interface(
    fn=qa_chatbot,
    inputs=gr.Textbox(lines=3, placeholder="Ask any question..."),
    outputs=gr.Textbox(label="Answer"),
    title="Local GPT Q&A Chatbot",
    description="Instruction-tuned Q&A bot, answers questions correctly.",
    flagging_mode="never"
)

# ----------------- Launch -----------------
iface.launch(
    server_name="127.0.0.1",
    server_port=free_port,
    inbrowser=True,
    prevent_thread_lock=True
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\Suraj Garole\Conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Suraj Garole\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP dow

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/google/flan-t5-large/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


model.safetensors:  15%|#4        | 461M/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

* Running on local URL:  http://127.0.0.1:62301
* To create a public link, set `share=True` in `launch()`.


In [46]:
# ----------------- Imports -----------------
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import socket

# ----------------- Load instruction-tuned model -----------------
model_name = "google/flan-t5-large"  # instruction-tuned for Q&A
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a HuggingFace pipeline for text generation
qa_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# ----------------- Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- Q&A Function -----------------
def qa_stream(user_input):
    # Prefix instruction for Q&A
    input_text = f"Question: {user_input} Answer:"
    # Generate text
    outputs = qa_pipeline(input_text, max_length=200, do_sample=True, top_p=0.9, temperature=0.7)
    answer = outputs[0]["generated_text"]
    return answer

# ----------------- Gradio Interface -----------------
iface = gr.Interface(
    fn=qa_stream,
    inputs=gr.Textbox(lines=3, placeholder="Ask any question..."),
    outputs=gr.Textbox(label="Answer"),
    title="Streaming GPT Q&A Bot",
    description="Instruction-tuned local chatbot that answers your questions accurately.",
    flagging_mode="never"
)

# ----------------- Launch -----------------
iface.launch(
    server_name="127.0.0.1",
    server_port=free_port,
    inbrowser=True,
    prevent_thread_lock=True
)


Device set to use cpu


* Running on local URL:  http://127.0.0.1:50967
* To create a public link, set `share=True` in `launch()`.


In [47]:
# ----------------- Imports -----------------
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import socket
import time

# ----------------- Load instruction-tuned model -----------------
# Using Flan-T5 large for general-purpose Q&A (GPT-like)
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

qa_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# ----------------- Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- Streaming Q&A Function -----------------
def gpt_bot(user_input):
    # Instruction prompt: acts like GPT
    input_text = f"You are a highly knowledgeable assistant. Answer the question clearly and accurately:\n{user_input}\nAnswer:"

    outputs = qa_pipeline(
        input_text,
        max_length=500,
        do_sample=True,        # allows diverse GPT-like responses
        temperature=0.7,       # controls creativity
        top_p=0.9
    )
    
    answer = outputs[0]["generated_text"]

    # Stream word by word
    words = answer.split()
    partial = ""
    for w in words:
        partial += w + " "
        yield partial
        time.sleep(0.02)  # streaming speed

# ----------------- Gradio Interface -----------------
iface = gr.Interface(
    fn=gpt_bot,
    inputs=gr.Textbox(lines=3, placeholder="Ask anything..."),
    outputs=gr.Textbox(label="Answer"),
    title="GPT-style Chatbot",
    description="A general-purpose GPT-like chatbot. Ask anything — science, engineering, history, coding, etc.",
    flagging_mode="never"
)

# ----------------- Launch -----------------
iface.launch(
    server_name="127.0.0.1",
    server_port=free_port,
    inbrowser=True,
    prevent_thread_lock=True
)


Device set to use cpu


* Running on local URL:  http://127.0.0.1:63852
* To create a public link, set `share=True` in `launch()`.


In [48]:
# ----------------- Imports -----------------
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import socket
import time

# ----------------- Load instruction-tuned model -----------------
# Using Flan-T5 large for general-purpose Q&A (GPT-like)
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

qa_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# ----------------- Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- Streaming Q&A Function -----------------
def gpt_bot(user_input):
    # Instruction prompt: acts like GPT
    input_text = f"You are a highly knowledgeable assistant. Answer the question clearly and accurately:\n{user_input}\nAnswer:"

    outputs = qa_pipeline(
        input_text,
        max_length=500,
        do_sample=True,        # allows diverse GPT-like responses
        temperature=0.7,       # controls creativity
        top_p=0.9
    )
    
    answer = outputs[0]["generated_text"]

    # Stream word by word
    words = answer.split()
    partial = ""
    for w in words:
        partial += w + " "
        yield partial
        time.sleep(0.02)  # streaming speed

# ----------------- Gradio Interface -----------------
iface = gr.Interface(
    fn=gpt_bot,
    inputs=gr.Textbox(lines=3, placeholder="Ask anything..."),
    outputs=gr.Textbox(label="Answer"),
    title="GPT-style Chatbot",
    description="A general-purpose GPT-like chatbot. Ask anything — science, engineering, history, coding, etc.",
    flagging_mode="never"
)

# ----------------- Launch -----------------
iface.launch(
    server_name="127.0.0.1",
    server_port=free_port,
    inbrowser=True,
    prevent_thread_lock=True
)


Device set to use cpu


* Running on local URL:  http://127.0.0.1:58076
* To create a public link, set `share=True` in `launch()`.


Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [49]:
# ----------------- Imports -----------------
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import socket
import time
import pyttsx3
import speech_recognition as sr
import threading

# ----------------- Load instruction-tuned model -----------------
# Using Flan-T5 large for general-purpose Q&A (GPT-like)
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

qa_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# ----------------- Initialize TTS engine -----------------
engine = pyttsx3.init()
def speak(text):
    def run():
        engine.say(text)
        engine.runAndWait()
    t = threading.Thread(target=run)
    t.start()

# ----------------- Initialize Speech Recognizer -----------------
recognizer = sr.Recognizer()
def listen_voice():
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        try:
            query = recognizer.recognize_google(audio)
            print("You said:", query)
            return query
        except sr.UnknownValueError:
            return "Sorry, I could not understand your speech."
        except sr.RequestError:
            return "Could not request results; check your internet connection."

# ----------------- Auto-find a free port -----------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = find_free_port()

# ----------------- Streaming Q&A Function -----------------
def gpt_bot(input_type, user_input):
    # Get query
    if input_type == "Voice":
        query = listen_voice()
    else:
        query = user_input

    # Instruction prompt
    input_text = f"You are a highly knowledgeable assistant. Answer the question clearly and accurately:\n{query}\nAnswer:"

    outputs = qa_pipeline(
        input_text,
        max_length=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    
    answer = outputs[0]["generated_text"]

    # Speak answer in background
    speak(answer)

    # Stream answer word by word
    words = answer.split()
    partial = ""
    for w in words:
        partial += w + " "
        yield partial
        time.sleep(0.02)  # streaming speed

# ----------------- Gradio Interface -----------------
iface = gr.Interface(
    fn=gpt_bot,
    inputs=[
        gr.Radio(["Text", "Voice"], label="Input Type"),
        gr.Textbox(lines=3, placeholder="Type your question (ignored if Voice is selected)")
    ],
    outputs=gr.Textbox(label="Answer"),
    title="GPT-style Chatbot (Voice + Text Input / Voice + Text Output)",
    description="Ask any question via text or voice. The answer will be displayed and spoken aloud.",
    flagging_mode="never"
)

# ----------------- Launch -----------------
iface.launch(
    server_name="127.0.0.1",
    server_port=free_port,
    inbrowser=True,
    prevent_thread_lock=True
)


Device set to use cpu


* Running on local URL:  http://127.0.0.1:63546
* To create a public link, set `share=True` in `launch()`.


Listening...


Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
